In [2]:
#@markdown # 🛠️ ติดตั้ง ComfyUI (เวอร์ชัน Shuttle3 GGUF พร้อมใช้งานทันที)
from IPython.display import Javascript , display, HTML
from google.colab import drive
import os, json
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 350})'''))

# เชื่อมต่อ Google Drive
%cd /content
drive.mount('/content/drive')

# ลบของเก่าออกก่อนถ้ามี
!rm -rf /content/ComfyUI

# ดาวน์โหลดเวอร์ชันล่าสุดของ ComfyUI (มี requirements.txt ครบ)
!git clone -b master https://github.com/comfyanonymous/ComfyUI.git
%cd /content/ComfyUI

# สร้าง user settings
os.makedirs("user/default", exist_ok=True)
with open("user/default/comfy.settings.json", "w") as f:
  json.dump({"Comfy.PreviewFormat": "webp;85"}, f, indent=2)

# ติดตั้ง cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# ติดตั้ง dependencies
!apt -y install -qq aria2
!pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu124 xformers!=0.0.18 triton SageAttention

# สร้างโฟลเดอร์สำหรับโมเดล
folders = ["checkpoints", "vae", "unet", "clip", "custom_nodes"]
for f in folders:
    os.makedirs(f"models/{f}", exist_ok=True)

# ✅ ดาวน์โหลดโมเดลหลักที่ workflow ใช้
!aria2c -x 8 -s 8 -d models/unet -o shuttle-3-diffusion-Q4_K_S.gguf https://huggingface.co/shuttleai/shuttle-3-diffusion-GGUF/resolve/main/shuttle-3-diffusion-Q4_K_S.gguf
!aria2c -x 8 -s 8 -d models/clip -o clip_l.safetensors https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors
!aria2c -x 8 -s 8 -d models/vae -o ae.safetensors https://huggingface.co/Patarapoom/model/resolve/main/ae.safetensors
!aria2c -x 8 -s 8 -d models/clip -o t5xxl_fp8_e4m3fn.safetensors https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors

# ✅ ติดตั้ง Custom Nodes ที่จำเป็น
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/kijai/ComfyUI-KJNodes
!git clone https://github.com/city96/ComfyUI-GGUF

# ✅ ติดตั้ง requirements ของ custom nodes
%cd /content/ComfyUI/custom_nodes/ComfyUI-KJNodes
!pip install -r requirements.txt

%cd /content/ComfyUI/custom_nodes/ComfyUI-GGUF
!pip install -r requirements.txt

# 🔙 กลับไปยังโฟลเดอร์หลัก
%cd /content/ComfyUI

# คัดลอกไฟล์ทั้งหมดจาก LoRAs ใน Google Drive ไปไว้ใน ComfyUI
!cp -r /content/drive/MyDrive/LoRAs/* /content/ComfyUI/models/loras/

# ✅ แจ้งสถานะ
display(HTML('<div style="padding:16px; background:#111; color:#0f0; font-size:18px; border-radius:12px; font-family:monospace">✅ ติดตั้งเสร็จแล้ว! พร้อมใช้งาน workflow Shuttle3 GGUF แล้วไปเซลล์ถัดไปรัน ComfyUI ได้เลย</div>'))


<IPython.core.display.Javascript object>

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'ComfyUI'...
remote: Enumerating objects: 19140, done.
remote: Counting objects: 100% (254/254), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 19140 (delta 199), reused 120 (delta 119), pack-reused 18886 (from 3)
Receiving objects: 100% (19140/19140), 67.88 MiB | 11.96 MiB/s, done.
Resolving deltas: 100% (12825/12825), done.
/content/ComfyUI
(Reading database ... 126411 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.4.0) over (2025.4.0) ...
Setting up cloudflared (2025.4.0) ...
Processing triggers for man-db (2.10.2-1) ...
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124

04/25 14:51:10 [NOTIC

In [ ]:
#@markdown # 🚀 เริ่ม ComfyUI พร้อม cloudflared
import subprocess, threading, time, socket
from IPython.display import Javascript, display
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

def tunnel(port=8188):
    while True:
        time.sleep(1)
        s = socket.socket()
        if s.connect_ex(("127.0.0.1", port)) == 0:
            break
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
                         stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        text = line.decode()
        if "trycloudflare.com " in text:
            url = text[text.find("http"):].strip()
            print("\n" + "="*60)
            print("📡  ComfyUI พร้อมใช้งานแล้วที่ลิงก์ด้านล่างนี้")
            print("🔗 ", url.strip())
            print("="*60 + "\n")

threading.Thread(target=tunnel, daemon=True).start()
!python main.py --dont-print-server --output-directory /content/drive/MyDrive/ComfyUI_Output


<IPython.core.display.Javascript object>

Setting output directory to: /content/drive/MyDrive/ComfyUI_Output
Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 52216 MB
pytorch version: 2.6.0+cu124
xformers version: 0.0.29.post3
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using xformers attention
Python version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
ComfyUI version: 0.3.30
ComfyUI frontend version: 1.17.11
[Prompt Server] web root: /usr/local/lib/python3.11/dist-packages/comfyui_frontend_package/static
Total VRAM 15095 MB, total RAM 52216 MB
pytorch version: 2.6.0+cu124
xformers version: 0.0.29.post3
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
ComfyUI-GGUF: Partial torch compile only, consider updating pytorch

Import times for custom nodes:
   0.0 seconds: /content/ComfyUI/custom_nodes/websocket_image_save.py
   0.0 seconds: /content/ComfyUI/custom_nodes/ComfyUI-GGUF
   0.1 seconds: /content/ComfyUI/custom_nodes/ComfyUI-KJNodes

